### Importing Libraries

In [35]:
import osgeo
from osgeo import gdal
import rasterio
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
for gpu in gpus:
    print(gpu)
import numpy as np
import matplotlib 
from matplotlib import pyplot as plt
import re
    
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, LeakyReLU,  Dense, Flatten, Reshape, Dropout, UpSampling2D, Lambda, BatchNormalization, Activation
from tensorflow.keras.preprocessing import image
from PIL import Image
import os

In [5]:
file_path = "images_downscaled\\April_7_2022.tif"
dataset = rasterio.open(file_path)

# Print general information
print("Raster Information:")
print(f"Number of Bands: {dataset.count}")
print(f"Width: {dataset.width}")
print(f"Height: {dataset.height}")
print(f"Coordinate Reference System (CRS): {dataset.crs}")
print(f"Transform (Affine): {dataset.transform}")

Raster Information:
Number of Bands: 4
Width: 256
Height: 256
Coordinate Reference System (CRS): None
Transform (Affine): | 1.00, 0.00, 0.00|
| 0.00, 1.00, 0.00|
| 0.00, 0.00, 1.00|


c:\Users\remne\AppData\Local\Programs\Python\Python311\Lib\site-packages\rasterio\__init__.py:319: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Downscaling the Images to a (256, 256) Dimension.

In [3]:
input_folder = "images"
output_folder = "images_downscaled"
target_resolution = (256, 256)

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate over the files in the input folder
for file_name in os.listdir(input_folder):
    # Check if the file is an image (you can modify the condition based on your image file extensions)
    if file_name.endswith(".tif"):
        # Construct the full paths to the input and output images
        input_image_path = os.path.join(input_folder, file_name)
        output_image_path = os.path.join(output_folder, file_name)

        # Open and resize the input image
        input_image = Image.open(input_image_path)
        resized_image = input_image.resize(target_resolution)

        # Save the resized image to the output folder
        resized_image.save(output_image_path)

Downscaling the Masks to a (256, 256) Dimension.

In [4]:
input_folder = "masks"
output_folder = "masks_downscaled"
target_resolution = (256, 256)

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate over the files in the input folder
for file_name in os.listdir(input_folder):
    # Check if the file is an image (you can modify the condition based on your image file extensions)
    if file_name.endswith(".tif"):
        # Construct the full paths to the input and output images
        input_image_path = os.path.join(input_folder, file_name)
        output_image_path = os.path.join(output_folder, file_name)

        # Open and resize the input image
        input_image = Image.open(input_image_path)
        resized_image = input_image.resize(target_resolution)

        # Save the resized image to the output folder
        resized_image.save(output_image_path)

In [36]:
# def build_generator():

#     model = Sequential()

#     model.add(Dense(256*256, input_dim = 100))
#     model.add(LeakyReLU(0.2))
#     model.add(Reshape((256, 256, 1))) # 1 is the number of channels

#     # Upsampling Block 1
#     model.add(UpSampling2D())
#     model.add(Conv2D(64, 3, padding='same'))
#     model.add(LeakyReLU(0.2))

#     # Upsampling Block 2
#     model.add(UpSampling2D())
#     model.add(Conv2D(64, 3, padding='same'))
#     model.add(LeakyReLU(0.2))

#     # Convolutional Block 1
#     model.add(Conv2D(64, 2, padding='same'))
#     model.add(LeakyReLU(0.2))

#     # Convolutuonal Block 2
#     model.add(Conv2D(64, 2, padding ='same'))
#     model.add(LeakyReLU(0.2))

#     # Convolutional layer to get to one channel
#     model.add(Conv2D(1, 1, padding='same', activation='sigmoid'))


#     return model

# def build_generator():
#     model = Sequential()

#     model.add(Dense(64 * 64 * 1, input_dim=100))
#     model.add(LeakyReLU(0.2))
#     model.add(Reshape((64, 64, 1)))

#     # Upsampling Block 1
#     model.add(UpSampling2D())
#     model.add(Conv2D(64, 3, padding='same'))
#     model.add(LeakyReLU(0.2))

#     # Upsampling Block 2
#     model.add(UpSampling2D())
#     model.add(Conv2D(64, 3, padding='same'))
#     model.add(LeakyReLU(0.2))

#     # Convolutional Block 1
#     model.add(Conv2D(64, 3, padding='same'))
#     model.add(LeakyReLU(0.2))

#     # Convolutional Block 2
#     model.add(Conv2D(64, 3, padding='same'))
#     model.add(LeakyReLU(0.2))

#     # # Convolutional layer to get to one channel
#     # model.add(Conv2D(1, 3, padding='same', activation='sigmoid'))

#     # Last layer of the generator
#     model.add(Conv2D(1, 3, padding='same', activation='sigmoid'))



#     return model


def build_generator():
    model = Sequential()

    model.add(Dense(128 * 64 * 64, activation='relu', input_dim=100))
    model.add(Reshape((64, 64, 128)))
    model.add(UpSampling2D())
    
    model.add(Conv2D(128, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    model.add(UpSampling2D())
    
    model.add(Conv2D(64, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    
    model.add(Conv2D(32, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    
    model.add(Conv2D(1, kernel_size=3, padding='same'))
    model.add(Activation('tanh'))

    noise = Input(shape=(100,))
    img = model(noise)

    # This is where you create the mask generator
    mask_model = Sequential()

    mask_model.add(Dense(128 * 64 * 64, activation='relu', input_dim=100))
    mask_model.add(Reshape((64, 64, 128)))
    mask_model.add(UpSampling2D())
    
    mask_model.add(Conv2D(128, kernel_size=3, padding='same'))
    mask_model.add(BatchNormalization(momentum=0.8))
    mask_model.add(Activation('relu'))
    mask_model.add(UpSampling2D())
    
    mask_model.add(Conv2D(64, kernel_size=3, padding='same'))
    mask_model.add(BatchNormalization(momentum=0.8))
    mask_model.add(Activation('relu'))
    
    mask_model.add(Conv2D(32, kernel_size=3, padding='same'))
    mask_model.add(BatchNormalization(momentum=0.8))
    mask_model.add(Activation('relu'))
    
    mask_model.add(Conv2D(1, kernel_size=3, padding='same'))
    mask_model.add(Activation('sigmoid'))

    mask = mask_model(noise)

    return model(noise, [img, mask])




generator = build_generator()

generator.summary()



TypeError: Exception encountered when calling layer 'batch_normalization' (type BatchNormalization).

unhashable type: 'list'

Call arguments received by layer 'batch_normalization' (type BatchNormalization):
  • inputs=tf.Tensor(shape=(None, 128, 128, 128), dtype=float32)
  • training=["<KerasTensor: shape=(None, 256, 256, 1) dtype=float32 (created by layer 'sequential_4')>", "<KerasTensor: shape=(None, 256, 256, 1) dtype=float32 (created by layer 'sequential_5')>"]
  • mask=None

In [32]:
# def build_discriminator():

#     model = Sequential()

#     # Convolutional Block 1
#     model.add(Conv2D(64, (3, 3), strides=(2, 2), input_shape=(256, 256, 1)))
#     model.add(LeakyReLU(0.2))
#     model.add(Dropout(0.25))

#     # Convolutional Block 2
#     model.add(Conv2D(128, (3, 3)))
#     model.add(LeakyReLU(0.2))
#     model.add(Dropout(0.25))

#     # Convolutional Block 3
#     model.add(Conv2D(256, (3, 3)))
#     model.add(LeakyReLU(0.2))
#     model.add(Dropout(0.25))

#     # Convolutional Block 4
#     model.add(Conv2D(512, (3, 3)))
#     model.add(LeakyReLU(0.2))
#     model.add(Dropout(0.25))

#     # Flatten and then Condense into 1
#     model.add(Flatten())
#     model.add(Dropout(0.2))
#     model.add(Dense(1, activation='sigmoid'))

#     return model


# def build_discriminator():
#     model = Sequential()

#     # Convolutional Block 1
#     model.add(Conv2D(64, (3, 3), strides=(2, 2), input_shape=(256, 256, 1)))
#     model.add(LeakyReLU(0.2))
#     model.add(Dropout(0.25))

#     # Convolutional Block 2
#     model.add(Conv2D(128, (3, 3)))
#     model.add(LeakyReLU(0.2))
#     model.add(Dropout(0.25))

#     # Convolutional Block 3
#     model.add(Conv2D(256, (3, 3)))
#     model.add(LeakyReLU(0.2))
#     model.add(Dropout(0.25))

#     # Convolutional Block 4
#     model.add(Conv2D(512, (3, 3)))
#     model.add(LeakyReLU(0.2))
#     model.add(Dropout(0.25))

#     # Flatten and then Condense into 1
#     model.add(Flatten())
#     model.add(Dropout(0.2))
#     model.add(Dense(1, activation='sigmoid'))

#     return model


from tensorflow.keras import Input
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.models import Model

def build_discriminator():
    # Input: Image
    img_input = Input(shape=(256, 256, 1))
    
    # Input: Mask
    mask_input = Input(shape=(256, 256, 1))
    
    # Concatenate image and mask along the channel dimension
    merged = Concatenate(axis=-1)([img_input, mask_input])

    # Convolutional Block 1
    x = Conv2D(64, (3, 3), strides=(2, 2))(merged)
    x = LeakyReLU(0.2)(x)
    x = Dropout(0.25)(x)

    # Convolutional Block 2
    x = Conv2D(128, (3, 3))(x)
    x = LeakyReLU(0.2)(x)
    x = Dropout(0.25)(x)

    # Convolutional Block 3
    x = Conv2D(256, (3, 3))(x)
    x = LeakyReLU(0.2)(x)
    x = Dropout(0.25)(x)

    # Convolutional Block 4
    x = Conv2D(512, (3, 3))(x)
    x = LeakyReLU(0.2)(x)
    x = Dropout(0.25)(x)

    # Flatten and then Condense into 1
    x = Flatten()(x)
    x = Dropout(0.2)(x)
    validity = Dense(1, activation='sigmoid')(x)

    model = Model([img_input, mask_input], validity)

    return model

In [27]:
discriminator = build_discriminator()

In [28]:
discriminator.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_2 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 concatenate (Concatenate)      (None, 256, 256, 2)  0           ['input_1[0][0]',                
                                                                  'input_2[0][0]']            

In [29]:
# Compile the discriminator
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])

In [20]:
from PIL import Image
import numpy as np
import os

def convert_to_grayscale(input_dir, output_dir):
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Get list of all file paths in the directory
    image_files = os.listdir(input_dir)

    # Iterate over each file
    for file in image_files:
        # Ensure the file is a .tif
        if file.endswith('.tif') or file.endswith('.tiff'):
            # Create full input file path
            input_file_path = os.path.join(input_dir, file)
            # Create full output file path
            output_file_path = os.path.join(output_dir, file)
            
            # Open the image file
            with Image.open(input_file_path) as img:
                # Convert the image to grayscale
                grayscale_img = img.convert("L")
                # Save the grayscale image to the output file
                grayscale_img.save(output_file_path)

# Call the function with your specific directories
convert_to_grayscale(input_dir='masks_downscaled', output_dir='masks_grayscale')



In [16]:
for mask in dataset.take(1):  # take one sample from the dataset
    print(mask)  # this should print the tensor representation of the mask


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]] [Op:IteratorGetNext]

In [21]:
# Get the list of image and mask files
image_folder = "images_downscaled"
mask_folder = "masks_grayscale"
image_files = os.listdir(image_folder)
mask_files = os.listdir(mask_folder)

# Set the number of training iterations
epochs = 100
batch_size = 8

In [22]:
def save_generated_images(epoch):
    # Generate fake images
    noise = np.random.normal(0, 1, (10, 100))
    fake_images = generator.predict(noise)

    # Create a directory to save the generated images
    save_dir = "generated_images"
    os.makedirs(save_dir, exist_ok=True)

    # Save the generated images
    for i, image in enumerate(fake_images):
        filename = f"generated_image_{epoch}_{i}.png"
        filepath = os.path.join(save_dir, filename)
        image = (image * 255).astype(np.uint8)
        Image.fromarray(image).save(filepath)

In [30]:
# Training loop
for epoch in range(epochs):
    # ---------------------
    # Train the discriminator
    # ---------------------
    # Select a random batch of images and masks
    idx = np.random.randint(0, len(image_files), batch_size)
    real_images = []
    real_masks = []
    for i in idx:
        image = np.array(Image.open(os.path.join(image_folder, image_files[i])))
        mask = np.array(Image.open(os.path.join(mask_folder, mask_files[i])))
        real_images.append(image)
        real_masks.append(mask)
    real_images = np.array(real_images) / 255.
    real_masks = np.array(real_masks) / 255.

    # Generate fake images and masks
    noise = np.random.normal(0, 1, (batch_size, 100))
    fake_images, fake_masks = generator.predict(noise)

    # Train the discriminator
    real_validity = discriminator.train_on_batch([real_images, real_masks], np.ones((batch_size, 1)))
    fake_validity = discriminator.train_on_batch([fake_images, fake_masks], np.zeros((batch_size, 1)))
    discriminator_loss = 0.5 * np.add(real_validity, fake_validity)

    # ---------------------
    # Train the generator
    # ---------------------
    noise = np.random.normal(0, 1, (batch_size, 100))
    # The generator wants the discriminator to label the generated samples
    # as real (ones)
    validity_y = np.array([1] * batch_size)

    # Train the generator
    generator_loss = combined_model.train_on_batch(noise, validity_y)

    # Print the progress
    print ("%d [D loss: %f] [G loss: %f]" % (epoch, discriminator_loss, generator_loss))

    # If at save interval => save generated image samples
    if epoch % save_interval == 0:
        save_imgs(epoch)


1/1 [==============================] - 0s 273ms/step


ValueError: too many values to unpack (expected 2)